In [ ]:
import sqlite3
from pathlib import Path

import pandas as pd

In [ ]:
DB_FILE = "../db_prod.sqlite3"

PROJECT_NAME_MAP = {
    "depends": "_data/dbs/multilang-depends/depends.db",
    "survey1": "_data/student_dbs/Team1_Final.db",
    "survey2": "_data/student_dbs/Team2_Final.db",
    "survey3": "_data/student_dbs/Team3_Final.db",
    "survey5": "_data/student_dbs/Team5_Final.db",
    "survey6": "_data/student_dbs/Team6_Final.db",
}

SEQUENCE_SQL = """
    SELECT
        S.name AS sequence,
        I.position,
        PA.name AS project_a,
        PB.name AS project_b,
        FA.filename AS filename_a,
        FB.filename AS filename_b,
        FA.content AS content_a,
        FB.content AS content_b
    FROM fileranker_sequenceitem I
    JOIN fileranker_sequence S ON S.id = I.sequence_id
    JOIN fileranker_file FA ON FA.id = I.file_a_id
    JOIN fileranker_file FB ON FB.id = I.file_b_id
    JOIN fileranker_project PA ON PA.id = FA.project_id
    JOIN fileranker_project PB ON PB.id = FB.project_id
    ORDER BY S.name, I.position
"""

RESPONSE_SQL = """
    SELECT
        S.name AS sequence,
        I.position,
        U.username,
        R.created_at AS responded_on,
        R.value
    FROM fileranker_response R
    JOIN auth_user U ON U.id = R.user_id
    JOIN fileranker_sequenceitem I ON I.id = R.item_id
    JOIN fileranker_sequence S ON S.id = I.sequence_id
    ORDER BY S.name, I.position, U.id
"""


def rename_project(name: str) -> str:
    if name in PROJECT_NAME_MAP:
        return PROJECT_NAME_MAP[name]
    return name


def fetch_seqs_df() -> pd.DataFrame:
    with sqlite3.connect(DB_FILE) as con:
        return pd.read_sql(SEQUENCE_SQL, con)

In [ ]:
with sqlite3.connect(DB_FILE) as con:
    seq_df = pd.read_sql(SEQUENCE_SQL, con)
    res_df = pd.read_sql(RESPONSE_SQL, con)

In [ ]:
seq_df["project_a"] = [rename_project(p) for p in seq_df["project_a"]]
seq_df["project_b"] = [rename_project(p) for p in seq_df["project_b"]]
seq_df

In [ ]:
seq_df = seq_df[seq_df["position"] < 1200]
seq_df

In [ ]:
seq_df.to_csv("../artifacts/my_sequences.csv", index=False)
res_df.to_csv("../artifacts/my_responses.csv", index=False)